In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AutoML SDK: AutoML image classification model


## Installation

Install the latest (preview) version of AutoML SDK.


In [ ]:
! pip3 install -U google-cloud-automl --user


Install the Google *cloud-storage* library as well.


In [ ]:
! pip3 install google-cloud-storage


### Restart the Kernel

Once you've installed the AutoML SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AutoML APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AutoML Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for AutoML. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with AutoML. Not all regions provide support for all AutoML services. For the latest support per region, see [Region support for AutoML services]()


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using AutoML Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an AutoML notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AutoML, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gcloud storage buckets create --location=$REGION gs://$BUCKET_NAME


Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gcloud storage ls --all-versions --long gs://$BUCKET_NAME


### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import AutoML SDK

Import the AutoML SDK into our Python environment.


In [ ]:
import json
import os
import sys
import time

from google.cloud import automl_v1beta1 as automl
from google.protobuf.json_format import MessageToJson, ParseDict

#### AutoML constants

Setup up the following constants for AutoML:

- `PARENT`: The AutoML location root path for dataset, model and endpoint resources.


In [ ]:
# AutoML location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

## Clients

The AutoML SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (AutoML).

You will use several clients in this tutorial, so set them all up upfront.

(?)


In [ ]:
def automl_client():
    return automl.AutoMlClient()


def prediction_client():
    return automl.PredictionServiceClient()


def operations_client():
    return automl.AutoMlClient()._transport.operations_client


clients = {}
clients["automl"] = automl_client()
clients["prediction"] = prediction_client()
clients["operations"] = operations_client()

for client in clients.items():
    print(client)

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/hmdb_split1.csv"

In [ ]:
! gcloud storage cat $IMPORT_FILE | head -n 10 


*Example output*:
```
TRAIN,gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv
TEST,gs://automl-video-demo-data/hmdb_split1_5classes_test_inf.csv
```


## Create a dataset


### [projects.locations.datasets.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/create)

#### Request


In [ ]:
dataset = {
    "display_name": "hmdb_" + TIMESTAMP,
    "video_classification_dataset_metadata": {},
}

print(
    MessageToJson(
        automl.CreateDatasetRequest(parent=PARENT, dataset=dataset).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "hmdb_20210228225744",
    "videoClassificationDatasetMetadata": {}
  }
}
```


#### Call


In [ ]:
request = clients["automl"].create_dataset(parent=PARENT, dataset=dataset)

#### Response


In [ ]:
result = request

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/VCN6574174086275006464",
  "displayName": "hmdb_20210228225744",
  "createTime": "2021-02-28T23:06:43.197904Z",
  "etag": "AB3BwFrtf0Yl4fgnXW4leoEEANTAGQdOngyIqdQSJBT9pKEChgeXom-0OyH7dKtfvA4=",
  "videoClassificationDatasetMetadata": {}
}
```


In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### [projects.locations.datasets.importData](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/importData)

#### Request


In [ ]:
input_config = {"gcs_source": {"input_uris": [IMPORT_FILE]}}

print(
    MessageToJson(
        automl.ImportDataRequest(
            name=dataset_short_id, input_config=input_config
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "name": "VCN6574174086275006464",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://automl-video-demo-data/hmdb_split1.csv"
      ]
    }
  }
}
```


#### Call


In [ ]:
request = clients["automl"].import_data(name=dataset_id, input_config=input_config)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result))

*Example output*:
```
{}
```


## Train a model


### [projects.locations.models.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/create)

#### Request


In [ ]:
model = {
    "display_name": "hmdb_" + TIMESTAMP,
    "dataset_id": dataset_short_id,
    "video_classification_model_metadata": {},
}

print(
    MessageToJson(automl.CreateModelRequest(parent=PARENT, model=model).__dict__["_pb"])
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "model": {
    "displayName": "hmdb_20210228225744",
    "datasetId": "VCN6574174086275006464",
    "videoClassificationModelMetadata": {}
  }
}
```


#### Call


In [ ]:
request = clients["automl"].create_model(parent=PARENT, model=model)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/VCN6188818900239515648"
}
```


In [ ]:
# The full unique ID for the training pipeline
model_id = result.name
# The short numeric ID for the training pipeline
model_short_id = model_id.split("/")[-1]

print(model_short_id)

## Evaluate the model


### [projects.locations.models.modelEvaluations.list](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/list)

#### Call


In [ ]:
request = clients["automl"].list_model_evaluations(parent=model_id)

#### Response


In [ ]:
import json

model_evaluations = [json.loads(MessageToJson(me.__dict__["_pb"])) for me in request]
# The evaluation slice
evaluation_slice = request.model_evaluation[0].name

print(json.dumps(model_evaluations, indent=2))

*Example output*
```
[
  {
    "name": "projects/116273516712/locations/us-central1/models/VCN6188818900239515648/modelEvaluations/1998146574672720266",
    "createTime": "2021-03-01T01:02:02.452298Z",
    "evaluatedExampleCount": 150,
    "classificationEvaluationMetrics": {
      "auPrc": 1.0,
      "confidenceMetricsEntry": [
        {
          "confidenceThreshold": 0.016075565,
          "recall": 1.0,
          "precision": 0.2,
          "f1Score": 0.33333334
        },
        {
          "confidenceThreshold": 0.017114623,
          "recall": 1.0,
          "precision": 0.202977,
          "f1Score": 0.3374578
        },
        
        # REMOVED FOR BREVITY
        
        {
          "confidenceThreshold": 0.9299338,
          "recall": 0.033333335,
          "precision": 1.0,
          "f1Score": 0.06451613
        }
      ]
    },
    "displayName": "golf"
  }
]
```


### [projects.locations.models.modelEvaluations.get](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/get)

#### Call


In [ ]:
request = clients["automl"].get_model_evaluation(name=evaluation_slice)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/VCN6188818900239515648/modelEvaluations/1998146574672720266",
  "createTime": "2021-03-01T01:02:02.452298Z",
  "evaluatedExampleCount": 150,
  "classificationEvaluationMetrics": {
    "auPrc": 1.0,
    "confidenceMetricsEntry": [
      {
        "confidenceThreshold": 0.016075565,
        "recall": 1.0,
        "precision": 0.2,
        "f1Score": 0.33333334
      },
      {
        "confidenceThreshold": 0.017114623,
        "recall": 1.0,
        "precision": 0.202977,
        "f1Score": 0.3374578
      },
      
      # REMOVED FOR BREVITY
      
      {
        "confidenceThreshold": 0.9299338,
        "recall": 0.006666667,
        "precision": 1.0,
        "f1Score": 0.013245033
      }
    ],
    "confusionMatrix": {
      "annotationSpecId": [
        "175274248095399936",
        "2048771693081526272",
        "4354614702295220224",
        "6660457711508914176",
        "8966300720722608128"
      ],
      "row": [
        {
          "exampleCount": [
            30,
            0,
            0,
            0,
            0
          ]
        },
        {
          "exampleCount": [
            0,
            30,
            0,
            0,
            0
          ]
        },
        {
          "exampleCount": [
            0,
            0,
            30,
            0,
            0
          ]
        },
        {
          "exampleCount": [
            0,
            0,
            0,
            30,
            0
          ]
        },
        {
          "exampleCount": [
            0,
            0,
            0,
            0,
            30
          ]
        }
      ],
      "displayName": [
        "ride_horse",
        "golf",
        "cartwheel",
        "pullup",
        "kick_ball"
      ]
    }
  }
}
```


## Make batch predictions


### Make the batch input file

To request a batch of predictions from AutoML Video, create a CSV file that lists the Cloud Storage paths to the videos that you want to annotate. You can also specify a start and end time to tell AutoML Video to only annotate a segment (segment-level) of the video. The start time must be zero or greater and must be before the end time. The end time must be greater than the start time and less than or equal to the duration of the video. You can also use inf to indicate the end of a video.

example:  
    `gs://my-videos-vcm/short_video_1.avi,0.0,5.566667`  
    `gs://my-videos-vcm/car_chase.avi,0.0,3.933333`  


In [ ]:
TRAIN_FILES = "gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv"

test_items = ! gcloud storage cat $TRAIN_FILES | head -n2

cols = str(test_items[0]).split(",")
test_item_1, test_label_1, test_start_1, test_end_1 = (
    str(cols[0]),
    str(cols[1]),
    str(cols[2]),
    str(cols[3]),
)
print(test_item_1, test_label_1)

cols = str(test_items[1]).split(",")
test_item_2, test_label_2, test_start_2, test_end_2 = (
    str(cols[0]),
    str(cols[1]),
    str(cols[2]),
    str(cols[3]),
)
print(test_item_2, test_label_2)

*Example output*:
```
gs://automl-video-demo-data/hmdb51/_Rad_Schlag_die_Bank__cartwheel_f_cm_np1_le_med_0.avi cartwheel
gs://automl-video-demo-data/hmdb51/Acrobacias_de_un_fenomeno_cartwheel_f_cm_np1_ba_bad_8.avi cartwheel
```


In [ ]:
import json

import tensorflow as tf

gcs_input_uri = "gs://" + BUCKET_NAME + "/test.csv"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = f"{test_item_1}, {test_start_1}, {test_end_1}"
    f.write(data + "\n")
    data = f"{test_item_2}, {test_start_2}, {test_end_2}"
    f.write(data + "\n")

print(gcs_input_uri)
! gcloud storage cat $gcs_input_uri

*Example output*:
```
gs://migration-ucaip-trainingaip-20210228225744/test.csv
gs://automl-video-demo-data/hmdb51/_Rad_Schlag_die_Bank__cartwheel_f_cm_np1_le_med_0.avi, 0.0, inf
gs://automl-video-demo-data/hmdb51/Acrobacias_de_un_fenomeno_cartwheel_f_cm_np1_ba_bad_8.avi, 0.0, inf
```


### [projects.locations.models.batchPredict](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/batchPredict)

#### Request


In [ ]:
input_config = {"gcs_source": {"input_uris": [gcs_input_uri]}}

output_config = {
    "gcs_destination": {"output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"}
}

batch_prediction = automl.BatchPredictRequest(
    name=model_id, input_config=input_config, output_config=output_config
)

print(MessageToJson(batch_prediction.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/VCN6188818900239515648",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://migration-ucaip-trainingaip-20210228225744/test.csv"
      ]
    }
  },
  "outputConfig": {
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210228225744/batch_output/"
    }
  }
}
```


#### Call


In [ ]:
request = clients["prediction"].batch_predict(request=batch_prediction)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:

```
{}
```


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_dataset = True
delete_model = True
delete_bucket = True

# Delete the dataset using the AutoML fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients["automl"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the AutoML fully qualified identifier for the model
try:
    if delete_model:
        clients["automl"].delete_model(name=model_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gcloud storage rm --recursive gs://$BUCKET_NAME